In [ ]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
gmx="gmx-k8s"
ntomp=2
ntmpi=2

In [ ]:
pdbfile='ranked_0.pdb'
pdbid='1L2Y'
afmodel='result_model_3.pkl'
atoms=[2,16,26,47,57,74,98,117,127,139,146,153,167,178,189,196,220,234,248,262]


In [ ]:
with open(afmodel, 'rb') as f:
  data = pickle.load(f)
bins = data['distogram']['bin_edges']
bins = np.append(bins,2*bins[-1]-bins[-2])
logits = data['distogram']['logits']
logits = np.where(logits > 50, 50, logits)
probs = np.exp(logits)/(1.0 + np.exp(logits))


In [ ]:
maxatom=304 # XXX: works for 1L2Y only

atom_str = ','.join(map(str,atoms))
bins_str = ','.join(map(str,bins))
with open('plumed.dat','w') as p:
    p.write(f"""
WHOLEMOLECULES ENTITY0=1-{maxatom}
ALPHA_FOLD ...
LABEL=afcv
ATOMS={atom_str}
LAMBDA=3
DISTANCES={bins_str}
""")
    for d in range(len(bins)):
        rows=[]
        for i in range(len(atoms)):
            rows.append(','.join(map(str,probs[i,:,d])))
        p.write(f"LOGIT_MATRIX{d}=%s\n" % (','.join(rows)))
    p.write("... ALPHA_FOLD\n")
    p.write("""
METAD ARG=afcv SIGMA=0.1 HEIGHT=1.0 FILE=HILLS PACE=1000 BIASFACTOR=15 TEMP=300 LABEL=mtd
PRINT ARG=afcv,mtd.bias STRIDE=100 FILE=COLVAR FMT=%8.4f
""")

In [ ]:
mdbox=1.5
!{gmx} pdb2gmx -f {pdbfile} -o {pdbid}.gro -water tip3p -ff amber94 -ignh && \
{gmx} editconf -f {pdbid}.gro -o {pdbid}-box.gro -c -d {mdbox} -bt dodecahedron && \
{gmx} solvate -cp {pdbid}-box.gro -cs spc216.gro -o {pdbid}-solv.gro -p topol.top && \
{gmx} grompp -f ions.mdp -c {pdbid}-solv.gro -p topol.top -o ions.tpr && \
{gmx} -i 13 genion -s ions.tpr -o {pdbid}-ions.gro -p topol.top -pname NA -nname CL -neutral

In [ ]:
!{gmx} grompp -f minim-sol.mdp -c {pdbid}-ions.gro -p topol.top -o em.tpr &&\
{gmx} -n {ntmpi} mdrun -v -deffnm em -ntomp {ntomp} -pin on &&\
{gmx} -i 10 energy -f em.edr -o em.xvg

In [ ]:
def read_xvg(fn):
    x = []
    y = []
    with open(fn) as fh:
        f = fh.readlines()

    for l in f:
        if l[0] != '#' and l[0] != '@':
            x1,y1 = l.split()
            x.append(float(x1))
            y.append(float(y1))

    return x,y

In [ ]:
x,y=read_xvg('em.xvg')

plt.figure(figsize=(15,5))
plt.plot(x,y)
plt.grid()
plt.xlabel('step')
plt.ylabel('potential (kJ/mol)')
plt.title('Energy minimization')

plt.show()

In [ ]:
!{gmx} grompp -f nvt.mdp -c em.gro -r em.gro -p topol.top -o nvt.tpr && \
{gmx} -n {ntmpi} mdrun -ntomp {ntomp}  -pin on -deffnm nvt && \
{gmx} -i 16 energy -f nvt.edr -o temp.xvg

In [ ]:
x,y=read_xvg('temp.xvg')
plt.figure(figsize=(15,5))
plt.plot(x,y)
plt.grid()
plt.xlabel('time (ps)')
plt.ylabel('temperature (K)')
plt.title('isothermal-isochoric equilibration')
plt.show()

In [ ]:
!{gmx} grompp -f npt.mdp -c nvt.gro -r nvt.gro -t nvt.cpt -p topol.top -o npt.tpr && \
unset OMP_NUM_THREADS && {gmx} -n {ntmpi} mdrun -ntomp {ntomp} -pin on -deffnm npt && \
{gmx} -i 18 energy -f npt.edr -o press.xvg && \
{gmx} -i 24 energy -f npt.edr -o dens.xvg

In [ ]:
xp,yp=read_xvg('press.xvg')
xd,yd=read_xvg('dens.xvg')

plt.figure(figsize=(15,8))
plt.subplot(211)
plt.plot(xp,yp)
plt.title('isothermal-isobaric equilibration')
plt.grid()
plt.ylabel("pressure (bar)")

plt.subplot(212)
plt.xlabel('time (ps)')
plt.ylabel('density (kg/m3)')
plt.grid()
plt.plot(xd,yd)
plt.show()

In [ ]:
mdsteps=100000 # 200 ps test run

In [ ]:
!cp md.mdp.template md.mdp
with open('md.mdp','a') as mdp:
    mdp.write(f"nsteps = {mdsteps}\n")

!{gmx} grompp -f md.mdp -c npt.gro -t npt.cpt -p topol.top -o md-vanilla.tpr

In [ ]:
!{gmx} -n {ntmpi} mdrun -ntomp {ntomp} -pin on -deffnm md-vanilla

In [ ]:
!{gmx} grompp -f md.mdp -c npt.gro -t npt.cpt -p topol.top -o md-af.tpr

In [ ]:
!{gmx} -n {ntmpi} mdrun -ntomp {ntomp} -pin on -deffnm md-af -plumed plumed.dat